In [2]:
from html.parser import HTMLParser

class UkrLibSiteParser(HTMLParser):
    composition_urls = []
    
    def parse_composition_urls(self, tag, attrs):
        if tag == "a":
            #print("TAG: ", tag)
            for attr in attrs:
                if attr[0] == 'href':
                    #print('http://ukrlit.org' + attr[1])
                    querystring = attr[1]
                    
                    if (not querystring.startswith('/tvory')) and (not querystring.startswith('http')):
                        #print(querystring)
                        querystring_parts = querystring.split('/')
                        #print(len(querystring_parts))
                        if (len(querystring_parts) is 3) and (not querystring_parts[2].startswith('tvory')):
                            author = querystring_parts[1]
                            author_parts = author.split('_')
                            composition = querystring_parts[2]
                            #print(composition)
                            
                            filename = author_parts[0] + "-" + composition + ".txt"
                            composition_url = 'http://ukrlit.org/faily/avtor/' + author + "/" + filename
                            #print(composition_url)
                            self.composition_urls.append([author, filename, composition_url])
        
    def handle_starttag(self, tag, attrs):
        self.parse_composition_urls(tag, attrs)
site_content = ''        
parser = UkrLibSiteParser()

In [3]:
import urllib.request
with urllib.request.urlopen('http://ukrlit.org/tvory/uk') as f:
    site_content = f.read().decode('utf-8')
    #print(site_content)
    

In [4]:
parser.feed(site_content)

print("Number of composition urls: ", len(parser.composition_urls))
#print(parser.composition_urls)

Number of composition urls:  2962


In [5]:
import shutil
from pathlib import Path

root_path_name = "data"
root_path = Path(root_path_name)

print("Root exits: ", root_path.exists() and root_path.is_dir())

if root_path.exists() and root_path.is_dir():
    print("-- removing root")
    shutil.rmtree(str(root_path))
    print("-- removed root")
    print("-- root exits: ", root_path.exists() and root_path.is_dir())
    
root_path.mkdir()
print("Root created: ", root_path.exists() and root_path.is_dir())

Root exits:  True
-- removing root
-- removed root
-- root exits:  False
Root created:  True


In [6]:
from collections import Counter
author_stats = Counter([x[0] for x in parser.composition_urls])

#print(author_stats)

# RESTORE POINT

In [7]:
def print_stats(stats):
    for item in sorted(stats):
        print(" - ", item[0].replace('_', ' ').title(), '|', item[1])

print("Authors in the database: ", len(author_stats))
print("Compositions: ", len(parser.composition_urls))

print("Most common:")
print_stats(sorted(author_stats.most_common(10)))

print("Full:")
print_stats(author_stats.items())

Authors in the database:  151
Compositions:  2962
Most common:
 -  Antonych Bohdan Ihor Vasyliovych | 226
 -  Hlibov Leonid Ivanovych | 184
 -  Hrinchak Vasyl Yakovych | 101
 -  Khvyliovyi Mykola | 120
 -  Rudanskyi Stepan Vasyliovych | 122
 -  Symonenko Vasyl Andriiovych | 149
 -  Vinhranovskyi Mykola Stepanovych | 272
 -  Voronko Platon Mykytovych | 118
 -  Zahul Dmytro Yuriiovych | 110
 -  Zerov Mykola Kostiantynovych | 83
Full:
 -  Afanasiev Chuzhbynskyi Oleksandr Stepanovych | 2
 -  Aleksandrov Volodymyr Stepanovych | 1
 -  Andiievska Emma | 5
 -  Andriiashyk Roman Vasyliovych | 2
 -  Andrukhovych Yurii Ihorovych | 5
 -  Andrusiak Ivan Mykhailovych | 7
 -  Antonenko Davydovych Borys Dmytrovych | 6
 -  Antonovych Volodymyr Bonifatiiovych | 7
 -  Antonych Bohdan Ihor Vasyliovych | 226
 -  Avramenko Serhii Volodymyrovych | 3
 -  Babii Stepan Oleksandrovych | 1
 -  Bahrianyi Ivan Pavlovych | 35
 -  Baidiuk Mykola | 10
 -  Bantyna Yaroslav | 1
 -  Baranov Viktor Fedorovych | 1
 -  Bark

In [13]:
for item in parser.composition_urls:
    author_path = root_path / item[0]
    
    if not author_path.exists():
        author_path.mkdir()
    
    file_path = author_path / item[1]
    
    req = urllib.request.urlretrieve(item[2], str(file_path))

Download files:
